In [ ]:
import pandas as pd
from datetime import datetime, date, time, timedelta
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
df = pd.read_csv("data/data_train_clean_withdummies.csv")

### Splitting the data

In [ ]:
#separate predictor variables
X = df.drop('FraudResult', axis =1)

# separate target variable
y = df['FraudResult']


In [ ]:
# importing train_test_split from sklearn
from sklearn.model_selection import train_test_split
# splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

### Data Balance - Oversampling

In [ ]:
from imblearn.over_sampling import SMOTE, ADASYN
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import recall_score, roc_auc_score, roc_curve